1. 문서의 내용을 읽는다. 
2. 문서를 쪼갠다. 
    - 토큰 수 초과로 답변을 생성하지 못할 수 있고 
    - 문서가 길면 (input 이 길면) 답변 생성이 오래걸림 
3. 임베딩 -> 벡터 데이터베이스에 저장 
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색 
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달 

In [1]:
%pip install --upgrade --quiet docx2txt langchain-community


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain_community.document_loaders import Docx2txtLoader

loader = Docx2txtLoader("./tax.docx")
document = loader.load()

In [7]:
document
len(document)

1

In [8]:
%pip install -qU langchain-text-splitter

ERROR: Could not find a version that satisfies the requirement langchain-text-splitter (from versions: none)
ERROR: No matching distribution found for langchain-text-splitter

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200
)

loader = Docx2txtLoader("./tax.docx")
document_list = loader.load_and_split(text_splitter=text_splitter)


In [21]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")


In [13]:
%pip install langchain-chroma


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [23]:
from dotenv import load_dotenv
from langchain_chroma import Chroma

database = Chroma.from_documents(documents=document_list, embedding=embeddings)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************J3EA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}